In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time 
import csv
import os
import scipy.stats as st
from pprint import pprint
from pandas import json_normalize
import seaborn as sns; sns.set()
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [2]:
#Jimmy Import data over time csv 

# import csv file that is a breakdown of cases and deaths per day by state and county
df = pd.read_csv("CSV Folder/us-counties.csv.txt")

#group by state and date so that data is no longer at the county level, but at the state level
state_df = df.groupby(["state","date"])

#Create a data frame that has total cases by date by state
cases = state_df["cases"].sum()
workable_df = pd.DataFrame(cases)
workable_df = workable_df.reset_index()

#get the data for deaths per state per day into a series format to add to the cases dataframe
deaths = state_df["deaths"].sum()
death=pd.DataFrame(deaths)
work_death=death.reset_index()
DEAD = work_death["deaths"]

# Add Deaths column to the cases dataframe
workable_df["Deaths"]=DEAD

#Create final dataframe with just the columns we are interested in
final_df = workable_df[["date","state","cases","Deaths"]]

#Sort final df by date and re-index
final_df = final_df.sort_values("date")
final_df = final_df.reset_index(drop = True)

#Create Data Frame to display increase in cases over time of top 10 States in total current cases
cases=final_df.pivot( index="date",columns="state",values = "cases")
cases = cases.fillna(0)
top_cases =cases[["New York", "New Jersey","Massachusetts","California","Illinois","Pennsylvania","Michigan","Florida","Connecticut","Louisiana"]]

#Create Data Frame to display increase in deaths over time of top 10 States in total current deaths
death_df = final_df[final_df["Deaths"]>0]
death_plot = death_df.pivot( index="date",columns="state",values = "Deaths")
top_deaths = death_plot[["New York", "New Jersey","Massachusetts","California","Illinois","Pennsylvania","Michigan","Florida","Connecticut","Louisiana"]]

In [3]:
#Liz's Hospital Dataframe
#read in .csv file
hospital_data = pd.read_csv("CSV Folder/hospitals.csv")

#set index to state_name
hospital_data.set_index("state_name",inplace=True)

#group dataframe by state_name
hg = hospital_data.groupby('state_name')

#do count of hg dataframe (where it groups by states)to get total count of hospitals, doctors, staff,
#icu beds, and icu beds utilized. Then add each series to new dataframe called 'state_hospital_data'

hg["num_icu_be"].count()
state_hospital_data = pd.DataFrame(hg["num_icu_be"].count())

hospitals = hg["hospital_n"].count()
state_hospital_data["Number of Hospitals"]=hospitals

doctors = hg["num_licens"].count()
state_hospital_data["Number of Doctors"]=doctors

staff = hg["num_staffe"].count()
state_hospital_data["Number of Staff"]=staff

icu_beds = hg["num_icu_be"].count()
state_hospital_data["ICU Beds Available"]=icu_beds

bed_utiliz = hg["bed_utiliz"].count()
state_hospital_data["ICU Beds Utilized"]=bed_utiliz

#drop double num_icu_be column
state_hospital_data.drop("num_icu_be",axis=1, inplace = True)

# Shweta State level data import
#reaching out to API to get current overall state data pertaining to cases, tests, hospitalizations, and deaths
url3 = "https://covidtracking.com/api/states"
response3 = requests.get(url3)
#Turned entire response into a dataframe
US_df = json_normalize(response3.json(), errors='ignore')
#Narrowed df to specific columns of interest 
US_df = US_df.iloc[:,[0,1,10,11,24,18,12,21]]

#Jimmy file for merging dataframes that have different state columns
#Read in a State Name Abbrev CSV so that df's could be joined with different state naming conventions
st_abrev = pd.read_csv("CSV Folder/Covid Analysis Data - StateNameAbrev.csv")

# Replacing the state abbreviation with names for joining to other data
tests_df = st_abrev.merge(US_df, how = 'inner',left_on = "Abbreviation",right_on = "state")

#Droping abbreviation column since no other dataset used abbrev.
tests_df.drop(columns = ["Abbreviation","state"],inplace = True)

#Joined hospital information with the general information
comb1_df= tests_df.merge(state_hospital_data,how = 'inner',left_on = "State",right_on = "state_name")
#comb1_df.drop(columns = "state_name", inplace = True)

#Sequoyah import of file on aggressive measures
#Read in file about rankings of aggressive measures limiting the virus spread
States_Most_Aggressive_Measures_df = pd.read_csv("CSV Folder/States with the Most Aggressive Measures in Limiting Virus Exposure.csv", encoding ='latin-1')

#Joined rankings data
comb2_df = comb1_df.merge(States_Most_Aggressive_Measures_df, how = 'inner', on = 'State' )

#Renamed Column Headers
comb2_df.rename(columns = {"positive":"Positive",
                           "negative":"Negative",
                           "pending":"Pending",
                           "totalTestResults": "Total Tested",
                           "recovered": "Recovered",
                           "hospitalizedCurrently":"Currently Hospitalized",
                           "death": "Total Deaths",
                           "num_icu_be":"ICU Beds",
                           "Prevention & Containment Rank": "Prevention & Containment Rank",
                           "Risk Factors & Infrastructure Rank": "Risk Factors & Infrastructure Rank",
                           "Economic Impact Rank": "Economic Impact Rank"}, inplace = True)
comb2_df

,State,Positive,Negative,Pending,Total Tested,Recovered,Currently Hospitalized,Total Deaths,Number of Hospitals,Number of Doctors,Number of Staff,ICU Beds Available,ICU Beds Utilized,Overall Rank,Prevention & Containment Rank,Risk Factors & Infrastructure Rank,Economic Impact Rank,Delta in Overall Rank vs. March 23
0,Alabama,5832,46863,NaN,52695,NaN,NaN,197.0,120,120,120,66,109,48,48,45,2,-15
1,Alaska,339,11942,NaN,12281,208.0,36.0,9.0,28,28,28,7,23,3,2,11,40,11
2,Arizona,6045,54669,NaN,60714,1313.0,639.0,266.0,126,126,126,46,108,39,36,46,19,-1
3,Arkansas,2741,32837,NaN,35578,929.0,101.0,45.0,106,106,106,40,100,46,45,49,5,1
4,California,39254,454919,NaN,494173,NaN,4880.0,1562.0,466,466,466,302,401,18,15,44,16,-17
5,Colorado,11262,41062,NaN,52324,NaN,1084.0,552.0,113,113,113,48,101,31,35,3,45,-22
6,Connecticut,23921,50117,NaN,74038,NaN,1877.0,1764.0,46,46,46,25,40,14,13,14,26,-8
7,Delaware,3442,13937,NaN,17379,703.0,277.0,100.0,16,16,16,5,15,15,14,29,28,-3
8,Florida,30174,286785,1287.0,316959,NaN,NaN,1031.0,276,276,276,172,250,44,46,24,32,-8
9,Georgia,22147,85029,NaN,107176,NaN,NaN,892.0,180,180,180,87,164,29,30,40,4,12


In [4]:
#Jimmy- Read in the Poverty Data File and Population Data File
df = pd.read_csv("CSV Folder/Poverty by state 2018.csv")
pop_df = pd.read_csv("CSV Folder/SCPRC-EST2019-18+POP-RES.csv")

#Select and rename columns of interest for merging
df.rename( columns={'Unnamed: 0':'State'}, inplace=True )
pop_df= pop_df[["NAME", "POPESTIMATE2019","POPEST18PLUS2019","PCNT_POPEST18PLUS"]]
pop_df = pop_df.rename(columns = {"NAME": "State","POPESTIMATE2019":"2019 Pop Est","POPEST18PLUS2019":"2019 18+","PCNT_POPEST18PLUS":"Percent 18+"})

#Merge the poverty and population df's and then merge with the state tests_df
poverty_df = pop_df.merge(df,how = 'inner',on = "State")
pov_death = tests_df.merge(poverty_df,how = 'inner', on = "State")

#Sort df, add categorical bins and append them to the dataframe
pov_death.sort_values("Percent in Poverty (All)", ascending = False).head()
bins = (0,10,11,12,13,14,15,16,17,20)
labels = ("< 10","10-10.99","11-11.99","12-12.99","13-13.99","14-14.99","15-15.99","16-16.99","> 17")
pov_death["Poverty Groups"] = pd.cut(x = pov_death["Percent in Poverty (All)"], bins = bins, labels = labels)
pov_death.rename(columns = {"death":"Deaths","positive":"Cases","negative":"Negative Tests","totalTestResults":"Total Tests"},inplace=True)

#Jimmy create and read in CSV of stay at home grades and append to df
grades = pd.read_csv("CSV Folder/StayatHomeGrades.txt")
grades.head()
pov_death = pov_death.merge(grades, how = 'inner', on = "State" )